In [ ]:
import requests
import time
import wget
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from elasticsearch_reader import ElasticsearchReader as esr
from elasticsearch_creator import ElasticsearchCreator as esc
# Geo Bounding CDMX
#-99.373327,19.0426,-98.933874,19.595002
np.random.seed(112358)
cdmx_bounding_box = {
    "top_left": {
        "lat": 19.595002,
        "lon": -99.373327
    },
    "bottom_right": {
        "lat": 19.0426,
        "lon": -98.933874
    }
}
def get_rand_position(bounding_box):
    lon = np.random.uniform(low=bounding_box['bottom_right']['lon'], high=bounding_box['top_left']['lon'])
    lat = np.random.uniform(low=bounding_box['bottom_right']['lat'], high=bounding_box['top_left']['lat'])
    return round(lon, 6), round(lat, 6)

def get_products_in_page(page):
    list_prod = page.findAll(attrs= {"vocab":"http://schema.org/","class": "row", "typeof": "Product"})
    list_prod = list_prod[0].findAll('div', attrs={'class':'row'})
    prods=[]
    for i in list_prod:
        url_image = i.findAll('img')[0]['src'].replace('ch/C','gr/G')
        title = i.findAll('img')[0]['alt']
        price = float(i.findAll('h2',attrs={"property": "price"})[0].text.strip("$ "))
        img_path = save_image(url_image, "img")
        lon, lat = get_rand_position(cdmx_bounding_box)
        prods.append({
            "title":title,
            "url_image": url_image,
            "price": price,
            "img_path":img_path,
            "place": "similares",
            "location": { 
                "lat": lat,
                "lon": lon
            },
            "created_at": datetime.now()
        })
    return prods

def get_products_with_delay(brower, max_delay):
    for i in range(max_delay):
        page = BeautifulSoup(browser.page_source,"html5lib")
        products = get_products_in_page(page)
        if products:
            break
        time.sleep(1)
    return products

def save_image(url, path_dir):
    url.split('/')[-1]
    return wget.download(url,out=path_dir)



In [ ]:
%%bash
#restart test
curl -XDELETE 'localhost:9200/inventory?pretty'
curl -XPUT 'localhost:9200/inventory?pretty'
rm /home/kira/projects/drugstore/img/*

In [ ]:
catalog_url = 'https://www.farmaciasdesimilares.com/#!/medicamentos'
browser = webdriver.Chrome('./chromedriver')
browser.get(catalog_url)
browser.implicitly_wait(10)

In [ ]:
for i in range(1,2): #82):
    products = get_products_with_delay(browser, 5)
    print("scrappend {} products in page: {}".format(len(products),i))
    browser.find_element_by_xpath('//*[@id="contenido"]/div/div[4]/div[1]/div/div[1]/dir-pagination-controls/ul/li[9]/a').click()
    actions_ = esc.get_actions("inventory", "document", products)
    result_list = esc.bulk_load(actions_)
    bulk_result = esc.process_result(result_list)
    print(bulk_result)
    time.sleep(1)


In [ ]:
%%bash
curl -XGET 'localhost:9200/inventory/document/_count?pretty'